In [1]:
import numpy as np
import termplotlib as tpl

from histogram_filter.world import Grid, Color, Agent, Environment, TransitionSpecification
from histogram_filter.utils import propagate, sense


### Setup


In [4]:
def setup():
    field = [Color.BLUE, Color.ORANGE, Color.BLUE, Color.BLUE, Color.ORANGE]
    my_grid = Grid(field)
    transition = TransitionSpecification(0, [0.05, 0.9, 0.05])
    my_agent = Agent(transition=transition, correct_measurement_prob=0.9)
    my_env = Environment(my_grid, my_agent, start_position=0)
    return my_env


def visualize_belief(belief):
    fig = tpl.figure()
    fig.plot(np.arange(len(belief)), belief, width=50, height=15)
    fig.show()


### Random Walk


In [5]:
my_env = setup()

num_steps = 10
for i in range(num_steps):
    if i > 0:
        my_env.move()
    print(f"Step: {i}")
    print(my_env)
    print(f"Sense: {my_env.sense().name}", "\n")


Step: 0
 X                 
Sense: BLUE 

Step: 1
 X                 
Sense: BLUE 

Step: 2
     X             
Sense: ORANGE 

Step: 3
         X         
Sense: BLUE 

Step: 4
         X         
Sense: BLUE 

Step: 5
             X     
Sense: BLUE 

Step: 6
                 X 
Sense: ORANGE 

Step: 7
 X                 
Sense: BLUE 

Step: 8
     X             
Sense: ORANGE 

Step: 9
         X         
Sense: BLUE 



### 1D Histogram Filter, without world interaction


In [6]:
my_env = setup()
belief = np.ones(len(my_env.grid)) / len(my_env.grid)
transition_matrix = my_env.get_transition_matrix()
sensor_matrix = my_env.get_sensor_matrix()
measurement_vector = [Color.ORANGE, Color.BLUE, Color.ORANGE]

for i, measurement in enumerate(measurement_vector):
    if i == 0:
        print("Grid:")
        print(my_env.grid, "\n")
        print("Initial belief:")
        print(belief, "\n")

    print(f"Step: {i + 1}")
    belief = propagate(belief, transition_matrix)
    print(f"After transition:")
    print([f'{b:.2f}' for b in belief])
    belief = sense(belief, measurement.value, sensor_matrix)
    print(f"After measurement {measurement.name}:")
    print([f'{b:.2f}' for b in belief], "\n")
    visualize_belief(belief)


Grid:
                    

Initial belief:
[0.2 0.2 0.2 0.2 0.2] 

Step: 1
After transition:
['0.20', '0.20', '0.20', '0.20', '0.20']
After measurement ORANGE:
['0.05', '0.43', '0.05', '0.05', '0.43'] 

  0.45 +--------------------------------------+
   0.4 |         **                           |
  0.35 |        *  *                         *|
       |       *    *                       * |
   0.3 |      *      *                     *  |
  0.25 |    **        **                 **   |
   0.2 |   *            *               *     |
  0.15 |  *              *             *      |
       | *                *           *       |
   0.1 |*                            *        |
  0.05 |                   **********         |
     0 +--------------------------------------+
       0   0.5   1   1.5   2  2.5   3   3.5   4
Step: 2
After transition:
['0.39', '0.09', '0.39', '0.07', '0.07']
After measurement BLUE:
['0.45', '0.01', '0.45', '0.08', '0.01'] 

   0.5 +------------------------------

### 1D Histogram Filter with world interaction


In [7]:
my_env = setup()
num_steps = 10
belief = np.ones(len(my_env.grid)) / len(my_env.grid)
transition_matrix = my_env.get_transition_matrix()
sensor_matrix = my_env.get_sensor_matrix()

for i in range(num_steps):
    if i == 0:
        print("Grid:")
        print(my_env, "\n")

        print(f"Step: {i}")
        print("Initial belief:")
        print(belief, "\n")
        visualize_belief(belief)

    print(f"Step: {i + 1}")
    print(f"After transition:")
    my_env.move()
    print(my_env)
    belief = propagate(belief, transition_matrix)
    print([f'{b:.2f}' for b in belief])

    measurement = my_env.sense()
    print(f"After measurement {measurement.name}:")
    belief = sense(belief, measurement.value, sensor_matrix)
    print([f'{b:.2f}' for b in belief], "\n")
    visualize_belief(belief)
    print("\n")


Grid:
 X                  

Step: 0
Initial belief:
[0.2 0.2 0.2 0.2 0.2] 

   0.202 +------------------------------------+
  0.2015 |                                    |
         |                                    |
   0.201 |                                    |
  0.2005 |                                    |
     0.2 |************************************|
         |                                    |
  0.1995 |                                    |
   0.199 |                                    |
         |                                    |
  0.1985 |                                    |
   0.198 +------------------------------------+
         0   0.5  1   1.5   2  2.5   3  3.5   4
Step: 1
After transition:
     X             
['0.20', '0.20', '0.20', '0.20', '0.20']
After measurement ORANGE:
['0.05', '0.43', '0.05', '0.05', '0.43'] 

  0.45 +--------------------------------------+
   0.4 |         **                           |
  0.35 |        *  *                         *|
